<a href="https://colab.research.google.com/github/Apekshaa2908/From-Clicks-to-Deliveries-Maximizing-E-commerce-Performance-with-Real-Time-Data-Integration/blob/main/Clicks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 4.4 MB/s eta 0:00:00


In [ ]:
## Table creation
import boto3

# Initialize a session using Amazon DynamoDB
session = boto3.Session(
    region_name='ap-southeast-2',
    aws_access_key_id='id',
    aws_secret_access_key='key'
)

# Initialize the DynamoDB resource
dynamodb = session.resource('dynamodb')

# Create the DynamoDB table
table = dynamodb.create_table(
    TableName='Clicks',
    KeySchema=[
        {
            'AttributeName': 'item_id',  # Partition Key
            'KeyType': 'HASH'  # Partition key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'item_id',  # Partition Key type
            'AttributeType': 'S'  # S = String
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

# Wait until the table exists
table.meta.client.get_waiter('table_exists').wait(TableName='Clicks')

print("Table created successfully!")


In [ ]:
## Generation of data
import boto3
import json
import random
import time
from datetime import datetime

# Creating a session for Kinesis
session = boto3.Session(
   region_name='ap-southeast-2',
    aws_access_key_id='id',
    aws_secret_access_key='key'
)

client = session.client('kinesis')

# Sample data to stimulate live stream
products = [
    {"item_id": "ITEM001", "item_name": "Mobile Phone"},
    {"item_id": "ITEM002", "item_name": "Laptop"},
    {"item_id": "ITEM003", "item_name": "Camera"}
]

# Adding click counts and timestamp to data
while True:
    for product in products:
        Click_Counts = random.randint(10, 100)
        payload = {
            "item_id": product["item_id"],
            "item_name": product["item_name"],
            "click_counts": Click_Counts,
            "timestamp": datetime.now().isoformat()
        }

        # Streaming the data into Kinesis
        response = client.put_record(
            StreamName='ClickstreamDataStream',
            PartitionKey=product["item_id"],  # Use the item ID as the partition key
            Data=json.dumps(payload).encode('utf-8')
        )

        print(f'Sent data to Kinesis: {payload}')